In [ ]:
import matplotlib.pyplot as plt
#import pyspeckit 

from astropy.io import fits
from astropy.wcs import WCS

from astropy.nddata import Cutout2D
from astropy.coordinates import SkyCoord
from astropy import units as u
import numpy as np

from spectral_cube import SpectralCube
from spectral_cube import Projection

from specutils import Spectrum1D
from specutils.manipulation import extract_region
from specutils.manipulation import noise_region_uncertainty
from specutils import Spectrum1D, SpectralRegion
from astropy.modeling import models
from specutils.fitting import find_lines_threshold
import specutils
from specutils.fitting import fit_lines

import regions


In [1]:
print('test')

test


In [ ]:
pos_cloudc1 = SkyCoord('17:46:21.3048683891', '-28:35:33.1211282499', unit=(u.hourangle, u.deg))
pos_cloudc2 = SkyCoord('17:46:18.3316118680', '-28:34:48.4717811920', unit=(u.hourangle, u.deg))
pos_cloudd = SkyCoord('17:46:22.6563371259', '-28:33:27.5405071803', unit=(u.hourangle, u.deg))
pos_filament = SkyCoord('17:46:20.9063719501', '-28:37:51.6942550990', unit=(u.hourangle, u.deg))

In [ ]:
rad_cloudc1 = 15*u.arcsec
rad_cloudc2 = 15*u.arcsec
rad_cloudd = 30*u.arcsec
rad_filament = 1.5*u.arcmin

In [ ]:
def plot_spectrum(spectrum, ax=None, **kwargs):
    if ax == None:
        ax = plt.subplot(111)
    ax.step(spectrum.spectral_axis.to(u.km/u.s), spectrum.flux, where='mid', **kwargs)
    ax.set_xlabel("Velocity ("+str(spectrum.spectral_axis.to(u.km/u.s).unit)+")")
    ax.set_ylabel("Brightness Temperature ("+str(spectrum.flux.unit)+")")
    return ax

In [ ]:
def get_cutout(filename, position, l, w):
    try: 
        hdu = fits.open(filename, ext='SCI')[0]
    except: 
        hdu = fits.open(filename)[0]
    data = hdu.data
    head = hdu.header

    ww = WCS(head)
    size = (l, w)

    cutout = Cutout2D(data, position=position, size=size, wcs=ww)
    return cutout

def get_cutout_405(position, l, w):
    fn = '/orange/adamginsburg/jwst/cloudc/images/F405_reproj_merged-fortricolor.fits'
    return get_cutout(fn, position, l, w)

def get_cutout_circ(position, rad):
    fn = '/orange/adamginsburg/jwst/cloudc/images/F405_reproj_merged-fortricolor.fits'
    return get_cutout(fn, position, 3*rad, 3*rad)

# Cloud C1 Spectrum

## HNCO

In [ ]:
spectrum_cloudc1 = Spectrum1D.read('/orange/adamginsburg/jwst/cloudc/alma/spectra/spectrum_cloudc1.fits')

In [ ]:
ax = plt.subplot(111)
ax.plot(spectrum_cloudc1.spectral_axis.to(u.km/u.s), spectrum_cloudc1.flux)

ax.set_xlabel('Velocity (km/s)')
ax.set_ylabel('Brightness Temperature (K)')

In [ ]:
spectrum = spectrum_cloudc1

In [ ]:
from matplotlib import pyplot as plt
plt.plot(spectrum.spectral_axis, spectrum.flux) 
plt.xlabel('Spectral Axis ({})'.format(spectrum.spectral_axis.unit)) 
plt.ylabel('Flux Axis({})'.format(spectrum.flux.unit)) 
plt.grid(True)

In [ ]:
noise_region = SpectralRegion(50*u.km/u.s, 99*u.km/u.s)
spectrum = noise_region_uncertainty(spectrum, noise_region)

In [ ]:
lines = find_lines_threshold(spectrum, noise_factor=3)  
lines[lines['line_type'] == 'emission']  

In [ ]:
from matplotlib import pyplot as plt
ax = plt.subplot(111)
ax.plot(spectrum.spectral_axis.to(u.km/u.s), spectrum.flux) 
ax.set_xlabel('Spectral Axis ({})'.format(spectrum.spectral_axis.to(u.km/u.s).unit)) 
ax.set_ylabel('Flux Axis({})'.format(spectrum.flux.unit)) 

for line in lines:
    ax.axvline(line['line_center'].to(u.km/u.s).value, color='k', linestyle='--')

ax.set_xlim(left=0, right=75)

In [ ]:
g_init = models.Gaussian1D(amplitude=2.5*u.K, mean=40*u.km/u.s, stddev=10*u.km/u.s)
g_fit = fit_lines(spectrum, g_init)
y_fit = g_fit(spectrum.spectral_axis)

In [ ]:
cutout_c1 = get_cutout_circ(pos_cloudc1, rad_cloudc1)
reg_c1 = regions.CircleSkyRegion(center=pos_cloudc1, radius=rad_cloudc1)

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax = plt.subplot(121)
ax.plot(spectrum.spectral_axis.to(u.km/u.s), spectrum.flux, label='Original Spectrum') 
ax.plot(spectrum.spectral_axis.to(u.km/u.s), y_fit, label='Gaussian Fit')
ax.set_xlabel('Spectral Axis ({})'.format(spectrum.spectral_axis.to(u.km/u.s).unit)) 
ax.set_ylabel('Flux Axis({})'.format(spectrum.flux.unit)) 

ax.set_xlim(left=0, right=75)
ax.legend()
ax.set_title('HNCO 4-3')


ax1 = plt.subplot(122, projection=cutout_c1.wcs)
ax1.imshow(cutout_c1.data, cmap='Greys', vmin=0, vmax=25)
pix_reg = reg_c1.to_pixel(wcs=cutout_c1.wcs)
pix_reg.plot(color='r')
ax1.set_xlabel("Right Ascension")
ax1.set_ylabel("Declination")

plt.tight_layout()

In [ ]:
g_fit

In [ ]:
specutils.analysis.fwhm(Spectrum1D(flux=y_fit, spectral_axis=spectrum.spectral_axis)).to(u.km/u.s)

# Cloud C2 Spectrum

In [ ]:
spectrum_cloudc2 = Spectrum1D.read('/orange/adamginsburg/jwst/cloudc/alma/spectra/spectrum_cloudc2.fits')

In [ ]:
ax = plot_spectrum(spectrum_cloudc2)

In [ ]:
noise_region = SpectralRegion(50*u.km/u.s, 99*u.km/u.s)
spectrum = noise_region_uncertainty(spectrum_cloudc2, noise_region)

In [ ]:
lines = find_lines_threshold(spectrum, noise_factor=3)  
lines[lines['line_type'] == 'emission']  

In [ ]:
ax = plot_spectrum(spectrum_cloudc2)
for line in lines:
    ax.axvline(line['line_center'].to(u.km/u.s).value, color='k', linestyle='--')

ax.set_xlim(left=-50, right=75)

In [ ]:
g1_init = models.Gaussian1D(amplitude=2.5*u.K, mean=10*u.km/u.s, stddev=5*u.km/u.s)
g2_init = models.Gaussian1D(amplitude=1.5*u.K, mean=0*u.km/u.s, stddev=5*u.km/u.s)
g3_init = models.Gaussian1D(amplitude=1.4*u.K, mean=-10*u.km/u.s, stddev=5*u.km/u.s)
g4_init = models.Gaussian1D(amplitude=1.4*u.K, mean=10*u.km/u.s, stddev=20*u.km/u.s)
g1234_fit = fit_lines(spectrum_cloudc2, g1_init+g2_init+g3_init+g4_init)
y_fit = g1234_fit(spectrum_cloudc2.spectral_axis)

In [ ]:
cutout_c2 = get_cutout_circ(pos_cloudc2, rad_cloudc2)
reg_c2 = regions.CircleSkyRegion(center=pos_cloudc2, radius=rad_cloudc2)

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax = plt.subplot(121)
ax = plot_spectrum(spectrum_cloudc2, ax=ax, label='Original Spectrum')
plot_spectrum(Spectrum1D(flux=y_fit, spectral_axis=spectrum_cloudc2.spectral_axis), ax=ax, label='Gaussian Fit')

ax.set_xlim(left=-50, right=75)
ax.legend()
ax.set_title('HNCO 4-3')


ax1 = plt.subplot(122, projection=cutout_c2.wcs)
ax1.imshow(cutout_c2.data, cmap='Greys', vmin=0, vmax=25)
pix_reg = reg_c2.to_pixel(wcs=cutout_c2.wcs)
pix_reg.plot(color='r')
ax1.set_xlabel("Right Ascension")
ax1.set_ylabel("Declination")

plt.tight_layout()

# Spectrum Cloud D

In [ ]:
spectrum_cloudd = Spectrum1D.read('/orange/adamginsburg/jwst/cloudc/alma/spectra/spectrum_cloudd.fits')

In [ ]:
ax = plot_spectrum(spectrum_cloudd)

In [ ]:
noise_region = SpectralRegion(50*u.km/u.s, 99*u.km/u.s)
spectrum = noise_region_uncertainty(spectrum_cloudd, noise_region)

In [ ]:
lines = find_lines_threshold(spectrum, noise_factor=3)  
lines[lines['line_type'] == 'emission']  

In [ ]:
ax = plot_spectrum(spectrum_cloudd)
for line in lines:
    ax.axvline(line['line_center'].to(u.km/u.s).value, color='k', linestyle='--')

ax.set_xlim(left=-50, right=75)

In [ ]:
g1_init = models.Gaussian1D(amplitude=2*u.K, mean=25*u.km/u.s, stddev=5*u.km/u.s)
g2_init = models.Gaussian1D(amplitude=1.7*u.K, mean=18*u.km/u.s, stddev=10*u.km/u.s)
g3_init = models.Gaussian1D(amplitude=0.5*u.K, mean=0*u.km/u.s, stddev=10*u.km/u.s)
#g4_init = models.Gaussian1D(amplitude=1.25*u.K, mean=15*u.km/u.s, stddev=10*u.km/u.s)
g1234_fit = fit_lines(spectrum_cloudd, g1_init+g2_init+g3_init)#+g4_init)
y_fit = g1234_fit(spectrum_cloudd.spectral_axis)

In [ ]:
cutout_d = get_cutout_circ(pos_cloudd, rad_cloudd)
reg_d = regions.CircleSkyRegion(center=pos_cloudd, radius=rad_cloudd)

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax = plt.subplot(121)
ax = plot_spectrum(spectrum_cloudd, ax=ax, label='Original Spectrum')
plot_spectrum(Spectrum1D(flux=y_fit, spectral_axis=spectrum_cloudd.spectral_axis), ax=ax, label='Gaussian Fit')

ax.set_xlim(left=-50, right=75)
ax.legend()
ax.set_title('HNCO 4-3')

ax1 = plt.subplot(122, projection=cutout_d.wcs)
ax1.imshow(cutout_d.data, cmap='Greys', vmin=0, vmax=25)
pix_reg = reg_d.to_pixel(wcs=cutout_d.wcs)
pix_reg.plot(color='r')
ax1.set_xlabel("Right Ascension")
ax1.set_ylabel("Declination")


plt.tight_layout()

In [ ]:
g1234_fit.mean_0, g1234_fit.mean_1, g1234_fit.mean_2#, g1234_fit.mean_3

In [ ]:
g1234_fit.stddev_0*2, g1234_fit.stddev_1*2, g1234_fit.stddev_2*2#, g1234_fit.stddev_3

# Filament

## CO

In [ ]:
spectrum_filament_co = Spectrum1D.read('/orange/adamginsburg/jwst/cloudc/alma/spectra/spectrum_filament_12CO-BEARS.fits')

In [ ]:
ax = plot_spectrum(spectrum_filament_co, label='12CO-BEARS')
ax.set_xlim(left=-100, right=0)

In [ ]:
fig = plt.figure(figsize=(6, 5))#, dpi=250)
ax = plt.subplot(111)
ax = plot_spectrum(spectrum_filament_co, label=r'$^{12}$CO 1-0', ax=ax)
#ax.set_xlim(left=-100, right=100)
ax.axvspan(-62, -50, color='gray', alpha=0.3, label='Filament')
ax.legend()
plt.tight_layout()

#plt.savefig('/orange/adamginsburg/jwst/cloudc/figures/filament_spectrum_12CO.pdf', bbox_inches='tight')

In [ ]:
g1_init = models.Gaussian1D(amplitude=8*u.K, mean=-55*u.km/u.s, stddev=1*u.km/u.s)
g1234_fit = fit_lines(spectrum_filament_co, g1_init)
y_fit = g1234_fit(spectrum_filament_co.spectral_axis)

In [ ]:
fig = plt.figure(figsize=(6, 5))
ax = plt.subplot(111)
ax = plot_spectrum(spectrum_filament_co, ax=ax, label='Original Spectrum')
plot_spectrum(Spectrum1D(flux=y_fit, spectral_axis=spectrum_filament_co.spectral_axis), ax=ax, label='Gaussian Fit')

ax.set_xlim(left=-100, right=0)
ax.legend()
ax.set_title('Filament CO 1-0')

In [ ]:
fig = plt.figure(figsize=(6, 5))
ax = plt.subplot(111)
ax = plot_spectrum(spectrum_filament_co, ax=ax, label=r'$^{12}$CO 2-1')
plot_spectrum(Spectrum1D(flux=y_fit, spectral_axis=spectrum_filament_co.spectral_axis), ax=ax, label='Gaussian Fit')

#ax.set_xlim(left=-100, right=0)
ax.axvspan(-62, -50, color='gray', alpha=0.3, label='Filament')
ax.legend()
#ax.set_title('Filament CO 1-0')

In [ ]:
g1234_fit.fwhm

## HNCO

In [ ]:
spectrum_filament_hnco = Spectrum1D.read('/orange/adamginsburg/jwst/cloudc/alma/spectra/spectrum_filament.fits')

In [ ]:
ax = plot_spectrum(spectrum_filament_hnco, label='HNCO 4-3')
ax.set_xlim(left=-80, right=-20)
ax.set_ylim(top=0.05)

In [ ]:
g1_init = models.Gaussian1D(amplitude=0.2*u.K, mean=-55*u.km/u.s, stddev=1*u.km/u.s)
g1234_fit = fit_lines(spectrum_filament_hnco, g1_init)
y_fit = g1234_fit(spectrum_filament_hnco.spectral_axis)

In [ ]:
fig = plt.figure(figsize=(6, 5))
ax = plt.subplot(111)
ax = plot_spectrum(spectrum_filament_hnco, ax=ax, label='Original Spectrum')
plot_spectrum(Spectrum1D(flux=y_fit, spectral_axis=spectrum_filament_hnco.spectral_axis), ax=ax, label='Gaussian Fit')

ax.set_xlim(left=-80, right=-20)
ax.set_ylim(top=0.05)
ax.legend()
ax.set_title('Filament HNCO 4-3')

In [ ]:
g1234_fit.fwhm